In [1]:
import os
import json
import torch
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
model_name = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct" #"nlpai-lab/KULLM3" #"rtzr/ko-gemma-2-9b-it"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True 
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ExaoneForCausalLM(
  (transformer): ExaoneModel(
    (wte): Embedding(102400, 4096, padding_idx=0)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x ExaoneBlock(
        (ln_1): ExaoneRMSNorm()
        (attn): ExaoneAttention(
          (attention): ExaoneSelfAttention(
            (rotary): ExaoneRotaryEmbedding()
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
        )
        (ln_2): ExaoneRMSNorm()
        (mlp): ExaoneGatedMLP(
          (c_fc_0): Linear(in_features=4096, out_features=14336, bias=False)
          (c_fc_1): Linear(in_features=4096, out_features=14336, bias=False)
          (c_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act): SiLU

In [3]:
with open("../data/filtered_exaone_restored.csv", "r") as f:
    df = pd.read_csv(f)

In [4]:
def inference_topic(text: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 입력으로 같은 target 라벨을 가지고 있는 'ID' 'text' 'target'으로 이루어진 텍스트들 여러 개가 들어올 겁니다. 해당 입력의 target의 공통 주제 단 한가지를 들어온 텍스트들의 분석을 통해 유추하시오. 유추할 때는 모든 텍스트들을 고려하여서 가장 포괄적인 키워드를 뽑아야합니다. 키워드는 뉴스 카테고리 중 하나입니다. 설명을 붙이지 말고 유추한 키워드 단 한 개만을 출력하세요."},
    {"role": "user", "content": text}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=256,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    result = generated_text.split("[|assistant|]")[-1].strip()
    if '\n' in result:
        result = result.split("\n")[0]
    print(result)
    del inputs, outputs, generated_text
        
    return result

In [5]:
def filter(text: str, subject: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 입력으로 같은 target 라벨을 가지고 있는 'ID' 'text' 'target'으로 이루어진 텍스트들 여러 개가 들어오고 그들의 공통 주제가 주어집니다. 들어온 텍스트들 중에는 공통 주제에 맞지 않는 텍스트들이 존재하는데 이들을 필터링하는 임무를 부여받았습니다. 당신은 주어진 텍스트들에서 이들을 정제한 후 그 결과를 설명을 붙이지 말고 'ID','text','target' 형태로 출력하세요."},
    {"role": "user", "content": text},
    {"role": "user", "content": subject}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=4096,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(generated_text)

    result = generated_text.split("[|assistant|]")[-1].strip()
    del inputs, outputs, generated_text
        
    return result

In [30]:
from tqdm.notebook import tqdm

filtered_datas = []
topics_dict = []
for i in tqdm(range(0,7), desc="label"):
    rows = df[df['target'] == i].sample(frac=1).reset_index(drop=True)
    keywords = {}
    context = ""
    for k, row in tqdm(rows.iterrows(), desc="Filter mis-labels", total=len(rows)):
        context += f"{row['ID']},{row['text']},{row['target']}\n"
        if (k + 1) % 60 == 0 or k == len(rows) - 1:
            topic = inference_topic(context)
            sub_topic = topic.split(" ")
            for sub in sub_topic:
                keywords[sub] = keywords.get(sub, 0) + 1
            context = ""
    keywords = dict(sorted(keywords.items(), key=lambda item: item[1], reverse=True))
    topics_dict.append(keywords)
    # for key in keywords.keys():
    #     if key not in topics:
    #         topics.append(key)
    #         break

topics = []
for i, sub1 in enumerate(topics_dict):
    nan = []
    candidates = []
    can_val = []
    for k, sub2 in enumerate(topics_dict):
        if i == k: continue
        skip_outer = False
        for key1 in sub1.keys():
            if key1 in topics: continue
            if key1 not in sub2.keys():
                if key1 not in candidates:
                    candidates.append(key1)
                    can_val.append(sub1[key1])
                break
            if key1 in sub2.keys():
                if sub1[key1] < sub2[key1]: 
                    if k > i: nan.append(key1)
                    continue
                if key1 not in candidates:
                    candidates.append(key1)
                    can_val.append(sub1[key1])

    mx = -1
    fnl = None
    for v, can in enumerate(candidates):
        if can in nan: continue
        if mx < v: 
            mx = v
            fnl = can
    topics.append(fnl)


print(topics)

    


label:   0%|          | 0/7 [00:00<?, ?it/s]

Filter mis-labels:   0%|          | 0/233 [00:00<?, ?it/s]

날씨
문화
힐링
문화


Filter mis-labels:   0%|          | 0/228 [00:00<?, ?it/s]

스포츠
스포츠
축구
스포츠


...

기술
기술 혁신
기술 혁신
기술 혁신


Filter mis-labels:   0%|          | 0/230 [00:00<?, ?it/s]

경제
경제
경제
경제


Filter mis-labels:   0%|          | 0/222 [00:00<?, ?it/s]

정치
정치
국제 정치
정치
['문화', '스포츠', '정치', '사회', '기술', '경제', '국제']


In [36]:
def relabel(text: str, subject: str):
    messages = [
    {"role": "system", 
    "content": "당신은 신문 기자입니다. 입력으로 신문 기사가 한 개가 들어오고 후보 주제둘이 주어집니다. 각 주제들은 개행 문자로 구분이 됩니다. 입력으로 들어온 후보 주제들 중에서 가장 신문 기사에 어울리는 주제 단 한 개를 출력하세요. 설명을 붙이지 말고 주제 하나만을 출력하세요. 복수의 주제를 출력하면 안됩니다."},
    {"role": "user", "content": text},
    {"role": "user", "content": subject}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    outputs = llm.generate(
        inputs.to(device),
        max_new_tokens=4096,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

    result = generated_text.split("[|assistant|]")[-1].strip()
    if '\n' in result:
        result = result.split("\n")[0]
    del inputs, outputs, generated_text
        
    return result

In [ ]:
final_topic = ""
for subtopic in topics:
    final_topic += subtopic + "\n"

with open("../data/original_nanoised.csv", 'r') as f:
    df2 = pd.read_csv(f)

label_filtered = []
for i, row in tqdm(df2.iterrows(), desc="Rerabelling", total=len(df2)):
    result = relabel(row['text'], final_topic)
    idx = -1
    for k, subtopic in enumerate(topics):
        if subtopic == result:
            idx = k
            break
    label_filtered.append(
        {
            'ID': row['ID'],
            'text': row['text'],
            'target': idx
        }
    )
    

In [38]:
df3 = pd.DataFrame(label_filtered)
df3.to_csv("../data/nanoised_label_filtered.csv", index=False, encoding="utf-8-sig")

In [39]:
print(len(df3))

1181


: 